<a href="https://colab.research.google.com/github/Himabindugssn/Text-Classification-Models/blob/main/Sentence_classification_custom%2Bgoogle_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#http://ieeexplore.ieee.org.dtulibrary.remotexs.in/document/8314136 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *
import tensorflow.keras
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.math import l2_normalize
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading Dataset

After using GLOVE, you will find that certain words in your training data are not present in its vocab. These are typically replaced with same-shape zero vector, which essentially means you are 'sacrificing' the word as your input feature, which can potentially be important for correct prediction. Another way to deal with this is to train your own word embeddings, using your training data, so that the semantic relationship of your own training corpus can be better represented.

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv', error_bad_lines=True)

In [ ]:
df.shape

(50000, 2)

In [ ]:
(xtrain,ytrain),(xtest,ytest)=imdb.load_data(num_words=5000) 

17465344/17464789 [==============================] - 0s 0us/step


In [ ]:
print(type(xtrain),len(xtest))

<class 'numpy.ndarray'> 25000


In [ ]:
word_idx=imdb.get_word_index() #getting vocab from imdb data

1646592/1641221 [==============================] - 0s 0us/step


In [ ]:
#len(word_idx.keys())

In [ ]:
!pip install nltk

In [ ]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word

In [ ]:
import nltk
nltk.download('stopwords')

#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [ ]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [ ]:
df['review']=df['review'].apply(denoise_text)


In [ ]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
    
#Apply function on review column
df['review']=df['review'].apply(remove_special_characters)

In [ ]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

#Apply function on review column
#df['review']=df['review'].apply(simple_stemmer)

In [ ]:
#set stopwords to english
stopword_list =set(stopwords.words('english'))

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

#Apply function on review column
df['review']= df['review'].apply(remove_stopwords)

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import time

from gensim.models import Word2Vec
from tqdm import tqdm

tqdm.pandas()

sentences = df['review']
train_sentences = list(sentences.progress_apply(str.split).values)

100%|██████████| 50000/50000 [00:00<00:00, 52389.90it/s]


In [ ]:


# Parameters reference : https://www.quora.com/How-do-I-determine-Word2Vec-parameters
# Feel free to customise your own embedding

#start_time = time.time()


#model = Word2Vec(sentences=train_sentences, 
 #                sg=1, 
 #                size=300,  
 #                workers=4)

#print(f'Time taken : {(time.time() - start_time) / 60:.2f} mins')

In [ ]:
#model.wv.vector_size

In [ ]:
#model.wv.vocab.keys()

In [ ]:
#model.wv.save_word2vec_format('custom_glove_100d.txt')

In [ ]:
maxlen=500
vocab_size=5000
emb_dimension=300
xtrain=pad_sequences(xtrain,maxlen=maxlen,padding='post')
xtest=pad_sequences(xtest,maxlen=maxlen,padding='post')

# Loading Google's pretrained word2vec model

In [ ]:
!wget -P download -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
from gensim.models import KeyedVectors


--2020-11-26 07:28:21--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.104.13
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.104.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘download/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  49.5MB/s    in 22s     

2020-11-26 07:28:44 (70.0 MB/s) - ‘download/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
model1 = KeyedVectors.load_word2vec_format('download/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
for i in model1.vocab.keys():
  train_sentences.append(i)

In [ ]:
len(train_sentences)

3050000

In [ ]:
import gensim
model = gensim.models.Word2Vec(train_sentences,size=300)

In [ ]:
#model1=KeyedVectors.load_word2vec_format('download/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
len(model.wv.vocab.keys())

51070

In [ ]:
model.wv.vector_size

300

In [ ]:
#model.intersect_word2vec_format(model1.vocab.keys())

In [ ]:
#model.intersect_word2vec_format('download/GoogleNews-vectors-negative300.bin.gz',lockf=1.0,binary=True)


In [ ]:
#model2=model.train(train_sentences,total_words=3050685,epochs=1)

# Embedding Layer

In [ ]:
embedding_matrix = np.zeros((vocab_size,emb_dimension))
for word, i in word_idx.items():
  if word in model.wv.vocab.keys() and i<vocab_size:  #if word in word2vec.vocab.keys() and i<vocab_size:
        embedding_matrix[i] = model.wv.word_vec(word) #embedding_matrix[i] = word2vec.word_vec(word) 

In [ ]:
embedding_matrix.shape

(5000, 300)

In [ ]:
np.save('drive/MyDrive/emb_custom.npy',embedding_matrix)

In [ ]:
embedding_matrix=np.load('drive/MyDrive/emb_custom.npy')

In [ ]:
xtrain=np.asarray(xtrain).astype(np.float32)
xtest=np.asarray(xtest).astype('float32')
ytrain=np.asarray(ytrain).astype('float32')
ytest=np.asarray(ytest).astype('float32')

train=tf.data.Dataset.from_tensor_slices((xtrain,ytrain))
test=tf.data.Dataset.from_tensor_slices((xtest,ytest))   

In [ ]:
train=train.batch(128)
test=test.batch(128)

# Model Architecture

In [ ]:
class MyModel(Model):

  def __init__(self,vocab_size,emb_dimension,embedding_matrix,filters,kernel_size):
    super(MyModel, self).__init__()
    self.emb=Embedding(vocab_size,emb_dimension,weights=[embedding_matrix],trainable=True)
    self.conv=Conv1D(filters=filters, kernel_size=kernel_size,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(l=0.01))
    self.bn=BatchNormalization()
    self.drop=Dropout(0.5)
    self.lstm=GRU(128)
    self.dense=Dense(1,activation='sigmoid')

  def call(self,input):
    x=self.emb(input)
    x=self.conv(x)
    x=self.bn(x)
    x=self.drop(x)    
    x=self.lstm(x)
    x=self.dense(x)
    return x

In [ ]:
model=MyModel(vocab_size=vocab_size,emb_dimension=emb_dimension,embedding_matrix=embedding_matrix,filters=128,kernel_size=7)

In [ ]:
train_acc_metric = tf.keras.metrics.BinaryAccuracy()
val_acc_metric = tf.keras.metrics.BinaryAccuracy()

In [ ]:
loss=tf.keras.losses.BinaryCrossentropy()
#optimizer=RMSprop(learning_rate=0.01)
epoch_losses = []
total_epochs=8

In [ ]:
def model_training():
  for epoch in range(total_epochs): 
    batch_losses=[] 
    for inputs, outputs in train:
      with tf.GradientTape() as tape:
        current_loss = loss(model(inputs), outputs) 
        grads=tape.gradient(current_loss, model.trainable_variables) 
      optimizer.apply_gradients(zip(grads, model.trainable_variables))
      batch_losses.append(current_loss) 
    
      train_acc_metric.update_state(outputs,model(inputs))
    
    for inputs, outputs in test:
      val_acc_metric.update_state(outputs,model(inputs))
  
    train_acc = train_acc_metric.result().numpy()
    train_acc_metric.reset_states()

    val_acc = val_acc_metric.result().numpy()
    val_acc_metric.reset_states()

    print("epoch ",epoch,", Training acc : " , train_acc,end="")
    print("  Validation acc: ",val_acc)

In [ ]:
#model_training()

In [ ]:
optimizer=RMSprop(learning_rate=0.005)
total_epochs=30
model_training()

epoch  0 , Training acc :  0.9282366  Validation acc:  0.8545998
epoch  1 , Training acc :  0.9314254  Validation acc:  0.8466518
epoch  2 , Training acc :  0.9356585  Validation acc:  0.864198
epoch  3 , Training acc :  0.93553895  Validation acc:  0.8596301
epoch  4 , Training acc :  0.9408801  Validation acc:  0.8654098
epoch  5 , Training acc :  0.9400431  Validation acc:  0.8560986
epoch  6 , Training acc :  0.94076055  Validation acc:  0.86757016
epoch  7 , Training acc :  0.9397959  Validation acc:  0.85546875
epoch  8 , Training acc :  0.9469388  Validation acc:  0.8686623
epoch  9 , Training acc :  0.9472178  Validation acc:  0.8691406
epoch  10 , Training acc :  0.9461815  Validation acc:  0.8660077
epoch  11 , Training acc :  0.9492905  Validation acc:  0.8676339
epoch  12 , Training acc :  0.95056605  Validation acc:  0.86850286
epoch  13 , Training acc :  0.9526786  Validation acc:  0.86589605
epoch  14 , Training acc :  0.9540258  Validation acc:  0.86728317
epoch  15 , T

In [ ]:
#optimizer=RMSprop(learning_rate=0.0001)
total_epochs=5
#model_training()

In [ ]:
#model.save_weights('drive/My Drive/Colab_files/model_100.h5')

In [ ]:
model.summary()